In [ ]:
import os
import json
import warnings
warnings.simplefilter('ignore')

import numpy as np

import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.exporters as sexp
import probeinterface as pi
import spikeinterface.widgets as sw

%matplotlib widget

In [ ]:
plx_file = os.getenv("PLX_FILE", "my_plexon_recording.plx")
print(f"Reading spike sorting from PLX_FILE: {plx_file}")

bin_file = os.getenv("BIN_FILE", "my_plexon_recording_converted.plx.bin")
print(f"Reading waveform data from BIN_FILE: {bin_file}")

ops_file = os.getenv("OPS_FILE", "my_kilosort_ops.json")
print(f"Reading Kilosort ops from OPS_FILE: {ops_file}")

out_dir = os.getenv("OUT_DIR", ".")
print(f"Writing results  to OUT_DIR: {out_dir}")

In [ ]:
print(f"Making a probe from Kilosort ops and chanMap.")

with open(ops_file) as f:
    ops = json.load(f)

probe = pi.Probe(ndim=2, si_units='um')
connected_indices = np.where(ops["chanMap"]["connected"])[0]
x_coords = np.array(ops["chanMap"]["xcoords"])[connected_indices]
y_coords = np.array(ops["chanMap"]["ycoords"])[connected_indices]
positions = np.vstack((x_coords, y_coords)).transpose()
probe.set_contacts(positions, shapes='circle', shape_params={'radius': 7.5})
probe.set_device_channel_indices(list(range(0, connected_indices.size)))

channel_ids = np.argsort(ops["chanMap"]["ycoords"])[connected_indices]
probe.set_contact_ids(channel_ids)

In [ ]:
print(f"Reading recording from {bin_file}")
recording = si.core.BinaryRecordingExtractor(
    bin_file,
    sampling_frequency=int(ops['fs']),
    num_chan=int(ops["NchanTOT"]),
    dtype="int16",
    gain_to_uV=1,
    offset_to_uV=0,
    is_filtered=True,
    time_axis=0
)
recording = recording.set_probe(probe)
print(recording)

In [ ]:
sw.plot_probe_map(recording)

In [ ]:
w = sw.plot_timeseries(
    {"raw": recording},
    clim=(-50, 50),
    time_range=[10, 11],
    order_channel_by_depth=True,
    backend="ipywidgets"
)

In [ ]:
print(f"Reading sorting from {plx_file}")
sorting = se.read_plexon_sorting(file_path=plx_file)
print(sorting)

In [ ]:
w_rs = sw.plot_rasters(sorting, time_range=[10, 11])

In [ ]:
waveform_folder = f"{out_dir}/waveforms"
print(f"Making a waveform extractor, extracting to {waveform_folder}.")
waveform_extractor = si.extract_waveforms(
    recording,
    sorting,
    folder=waveform_folder,
    overwrite=True,
    sparse=False)
print(waveform_extractor)

In [ ]:
phy_folder = f"{out_dir}/phy"
print(f"Exporting to phy in folder {phy_folder}.")
sexp.export_to_phy(
    waveform_extractor,
    phy_folder,
    copy_binary=False,
    remove_if_exists=True,
    compute_amplitudes=True,
    compute_pc_features=True
)